# BentoML Example: Fast AI with Tabular data

This notebook is based on fastai's cours v3 lesson 4.  We are going to train a model that predict salary range base on the data we provided.

[BentoML](http://bentoml.ai) is an open source platform for machine learning model serving and deployment. In this project we will use BentoML to package the trained fast.ai model, and build a containerized REST API model server.


![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=fast-ai&ea=fast-ai-salary-range-prediction&dt=fast-ai-salary-range-prediction)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
!pip install fastai
!pip install bentoml

In [2]:
from fastai.tabular import *

## Prepare Training Data

In [3]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')

In [4]:
dep_var = 'salary'
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [FillMissing, Categorify, Normalize]

In [5]:
test = TabularList.from_df(df.iloc[800:1000].copy(), path=path, cat_names=cat_names, cont_names=cont_names)

In [6]:
data = (TabularList.from_df(df, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(800,1000)))
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch())

In [7]:
data.show_batch(rows=10)

workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,target
Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,False,-0.4095,3.0509,-0.4224,<50k
Private,Some-college,Married-civ-spouse,Exec-managerial,Husband,White,False,0.5434,-1.1076,-0.0312,<50k
Self-emp-not-inc,Bachelors,Never-married,Sales,Not-in-family,White,False,0.4701,1.2268,1.1422,>=50k
Private,HS-grad,Married-civ-spouse,Farming-fishing,Husband,White,False,0.3235,0.0173,-0.4224,<50k
Private,HS-grad,Never-married,#na#,Not-in-family,White,True,-0.8493,-0.3763,-0.0312,<50k
Private,HS-grad,Never-married,Machine-op-inspct,Unmarried,Black,False,-0.8493,-0.4789,-0.4224,<50k
Private,Bachelors,Married-civ-spouse,Sales,Husband,Asian-Pac-Islander,False,-0.6294,1.0659,1.1422,<50k
Private,12th,Never-married,Other-service,Own-child,White,False,-1.5090,-0.5379,-0.8135,<50k
Self-emp-inc,10th,Never-married,Sales,Own-child,White,False,-1.5823,-0.0769,-1.5958,<50k
?,Some-college,Never-married,?,Own-child,Black,False,-1.4357,-0.1468,-0.0312,<50k


## Model Training

In [8]:
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)

In [9]:
learn.fit(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.365644,0.370601,0.840000,00:03


In [10]:
row = df.iloc[0] # sample input date for testing

learn.predict(row)

(Category >=50k, tensor(1), tensor([0.4036, 0.5964]))

## Create BentoService for model serving

In [11]:
%%writefile tabular_csv.py

from bentoml import env, api, artifacts, BentoService
from bentoml.artifact import FastaiModelArtifact
from bentoml.adapters import DataframeInput


@env(pip_dependencies=['fastai'])
@artifacts([FastaiModelArtifact('model')])
class FastaiTabularModel(BentoService):
    
    @api(input=DataframeInput())
    def predict(self, df):
        results = []
        for _, row in df.iterrows():       
            prediction = self.artifacts.model.predict(row)
            results.append(prediction[0].obj)
        return results

Overwriting tabular_csv.py


## Save BentoService to file archive

In [12]:
# 1) import the custom BentoService defined above
from tabular_csv import FastaiTabularModel

# 2) `pack` it with required artifacts
svc = FastaiTabularModel()
svc.pack('model', learn)

# 3) save your BentoSerivce
saved_path = svc.save()

[2020-02-14 12:57:51,895] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.6.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-02-14 12:57:52,324] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.6.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml

no previously-included directories found matching 'examples'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'docs'
no previously-included directories found matching 'scripts'


writing manifest file 'BentoML.egg-info/SOURCES.txt'
running check


creating BentoML-0.6.2+8.gd95a887
creating BentoML-0.6.2+8.gd95a887/BentoML.egg-info
creating BentoML-0.6.2+8.gd95a887/bentoml
creating BentoML-0.6.2+8.gd95a887/bentoml/artifact
creating BentoML-0.6.2+8.gd95a887/bentoml/bundler
creating BentoML-0.6.2+8.gd95a887/bentoml/cli
creating BentoML-0.6.2+8.gd95a887/bentoml/clipper
creating BentoML-0.6.2+8.gd95a887/bentoml/configuration
creating BentoML-0.6.2+8.gd95a887/bentoml/deployment
creating BentoML-0.6.2+8.gd95a887/bentoml/deployment/aws_lambda
creating BentoML-0.6.2+8.gd95a887/bentoml/deployment/sagemaker
creating BentoML-0.6.2+8.gd95a887/bentoml/handlers
creating BentoML-0.6.2+8.gd95a887/bentoml/marshal
creating BentoML-0.6.2+8.gd95a887/bentoml/migrations
creating BentoML-0.6.2+8.gd95a887/bentoml/migrations/versions
creating BentoML-0.6.2+8.gd95a887/bentoml/proto
creating BentoML-0.6.2+8.gd95a887/bentoml/repository
creating BentoML-0.6.2+8.gd95a887/bentoml/server
creating BentoML-0.6.2+8.gd95a887/bentoml/server/static
creating BentoML-0

copying bentoml/proto/yatai_service_pb2.py -> BentoML-0.6.2+8.gd95a887/bentoml/proto
copying bentoml/proto/yatai_service_pb2_grpc.py -> BentoML-0.6.2+8.gd95a887/bentoml/proto
copying bentoml/repository/__init__.py -> BentoML-0.6.2+8.gd95a887/bentoml/repository
copying bentoml/repository/metadata_store.py -> BentoML-0.6.2+8.gd95a887/bentoml/repository
copying bentoml/server/__init__.py -> BentoML-0.6.2+8.gd95a887/bentoml/server
copying bentoml/server/bento_api_server.py -> BentoML-0.6.2+8.gd95a887/bentoml/server
copying bentoml/server/bento_sagemaker_server.py -> BentoML-0.6.2+8.gd95a887/bentoml/server
copying bentoml/server/gunicorn_config.py -> BentoML-0.6.2+8.gd95a887/bentoml/server
copying bentoml/server/gunicorn_server.py -> BentoML-0.6.2+8.gd95a887/bentoml/server
copying bentoml/server/marshal_server.py -> BentoML-0.6.2+8.gd95a887/bentoml/server
copying bentoml/server/middlewares.py -> BentoML-0.6.2+8.gd95a887/bentoml/server
copying bentoml/server/utils.py -> BentoML-0.6.2+8.gd95a

## Use BentoService with BentoML CLI

**Use `bentoml get` to list all versions of BentoService, including the version tag will display additional information and metadata**

In [13]:
!bentoml get FastaiTabularModel

BENTO_SERVICE                             AGE            APIS                       ARTIFACTS
FastaiTabularModel:20200214125752_4055F5  36.61 seconds  predict<DataframeHandler>  model<FastaiModelArtifact>


In [14]:
!bentoml get FastaiTabularModel:20200214125752_4055F5

{
  "name": "FastaiTabularModel",
  "version": "20200214125752_4055F5",
  "uri": {
    "type": "LOCAL",
    "uri": "/Users/bozhaoyu/bentoml/repository/FastaiTabularModel/20200214125752_4055F5"
  },
  "bentoServiceMetadata": {
    "name": "FastaiTabularModel",
    "version": "20200214125752_4055F5",
    "createdAt": "2020-02-14T20:58:16.897556Z",
    "env": {
      "condaEnv": "name: bentoml-FastaiTabularModel\nchannels:\n- defaults\ndependencies:\n- python=3.7.3\n- pip\n",
      "pipDependencies": "bentoml==0.6.2\nfastai",
      "pythonVersion": "3.7.3"
    },
    "artifacts": [
      {
        "name": "model",
        "artifactType": "FastaiModelArtifact"
      }
    ],
    "apis": [
      {
        "name": "predict",
        "handlerType": "DataframeHandler",
        "docs": "BentoService API",
        "handlerConfig": {
          "orient": "records",
          "typ": "frame",
          "input_dtypes": null,
          "output_orient": "records"
        }
      }

  }
}


Quickly get prediction result with `bentoml run`

In [15]:
!bentoml run FastaiTabularModel:20200214125752_4055F5 predict \
    --input https://raw.githubusercontent.com/bentoml/gallery/master/fast-ai/salary-range-prediction/test.csv

[2020-02-14 12:59:25,233] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.6.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-02-14 12:59:25,245] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.6.2,  but loading from BentoML version 0.6.2+8.gd95a887
[2020-02-14 12:59:25,260] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be

Start a local realtime prediction service with `bentoml serve`

## Model Serving via REST API

*Note: Running as local rest api server does not work with Google Colab, please copy this notebook to run it locally*

In [ ]:
!bentoml serve {saved_path}

 * Serving Flask app "TabularModel" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Oct/2019 15:08:25] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2019 15:08:27] "POST /predict HTTP/1.1" 200 -


### Send prediction requeset to the REST API server

#### JSON Request

```bash
curl -X POST \
  http://localhost:5000/predict \
  -H 'Content-Type: application/json' \
  -d '[{
  "age": 49,
  "workclass": "Private",
  "fnlwgt": 101320,
  "education": "Assoc-acdm",
  "education-num": 12.0,
  "marital-status": "Married-civ-spouse",
  "occupation": "",
  "relationship": "Wift",
  "race": "White",
  "sex": "Female",
  "capital-gain": 0,
  "capital-loss": 1902,
  "hours-per-week": 40,
  "native-country": "United-States",
  "salary": ">=50k"
}]'
```

#### CSV Request

```bash
curl -X POST "http://127.0.0.1:5000/predict" \
    -H "Content-Type: text/csv" \
    --data-binary @test.csv
```

## Install saved BentoService as PyPI package

In [16]:
!pip install {saved_path}

Processing /Users/bozhaoyu/bentoml/repository/FastaiTabularModel/20200214125752_4055F5


  Created wheel for FastaiTabularModel: filename=FastaiTabularModel-20200214125752_4055F5-py3-none-any.whl size=253287 sha256=eefa4b0882d2f3cbffdc36823fe5cc70ccd31a9b60df07808b3ec59fd683d513
  Stored in directory: /private/var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/pip-ephem-wheel-cache-c5vwp5hd/wheels/0a/f1/25/06c81c2881a541c58f44c3e1f0d968c9a2bfbf281323a651d4
Successfully built FastaiTabularModel


In [17]:
!FastaiTabularModel info

[2020-02-14 13:01:34,234] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.6.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-02-14 13:01:34,248] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.6.2,  but loading from BentoML version 0.6.2+8.gd95a887
{
  "name": "FastaiTabularModel",
  "version": "20200214125752_4055F5",
  "created_at": "2020-02-14T20:58:16.897556Z",
  "env": {
    "conda_env": "name: bentoml-FastaiTabularModel\nchannels:\n- defaults\ndependencies:\n- python

In [18]:
# Use CSV data
!FastaiTabularModel run predict \
--input=https://raw.githubusercontent.com/bentoml/gallery/master/fast-ai/salary-range-prediction/test.csv

[2020-02-14 13:01:48,246] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.6.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-02-14 13:01:48,255] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.6.2,  but loading from BentoML version 0.6.2+8.gd95a887
[2020-02-14 13:01:48,257] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be

In [20]:
# Use json data
!FastaiTabularModel run predict \
--input=https://raw.githubusercontent.com/bentoml/gallery/master/fast-ai/salary-range-prediction/test.json

[2020-02-14 13:02:03,680] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.6.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-02-14 13:02:03,689] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.6.2,  but loading from BentoML version 0.6.2+8.gd95a887
[2020-02-14 13:02:03,691] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be

### Use BentoService with Docker

Use the auto generated `Dockerfile` inside the BentoService file bundle to build docker image

In [36]:
!cd {saved_path} && docker build -t fastai-salary .

Sending build context to Docker daemon  1.181MB
Step 1/12 : FROM continuumio/miniconda3:4.7.12
 ---> 406f2b43ea59
Step 2/12 : ENTRYPOINT [ "/bin/bash", "-c" ]
 ---> Running in feff1870c1ef
Removing intermediate container feff1870c1ef
 ---> 5554f67fe14d
Step 3/12 : EXPOSE 5000
 ---> Running in fda6256732f5
Removing intermediate container fda6256732f5
 ---> 138d488f26e6
Step 4/12 : RUN set -x      && apt-get update      && apt-get install --no-install-recommends --no-install-suggests -y libpq-dev build-essential      && rm -rf /var/lib/apt/lists/*
 ---> Running in 5a66ee5072a7
+ apt-get update
Get:1 http://security.debian.org/debian-security buster/updates InRelease [65.4 kB]
Get:2 http://deb.debian.org/debian buster InRelease [122 kB]
Get:3 http://deb.debian.org/debian buster-updates InRelease [49.3 kB]
Get:4 http://security.debian.org/debian-security buster/updates/main amd64 Packages [176 kB]
Get:5 http://deb.debian.org/debian buster/main amd64 Packages [7907 kB]
Fetched 8319 kB in 3s

Selecting previously unselected package libatomic1:amd64.
Preparing to unpack .../15-libatomic1_8.3.0-6_amd64.deb ...
Unpacking libatomic1:amd64 (8.3.0-6) ...
Selecting previously unselected package libasan5:amd64.
Preparing to unpack .../16-libasan5_8.3.0-6_amd64.deb ...
Unpacking libasan5:amd64 (8.3.0-6) ...
Selecting previously unselected package liblsan0:amd64.
Preparing to unpack .../17-liblsan0_8.3.0-6_amd64.deb ...
Unpacking liblsan0:amd64 (8.3.0-6) ...
Selecting previously unselected package libtsan0:amd64.
Preparing to unpack .../18-libtsan0_8.3.0-6_amd64.deb ...
Unpacking libtsan0:amd64 (8.3.0-6) ...
Selecting previously unselected package libubsan1:amd64.
Preparing to unpack .../19-libubsan1_8.3.0-6_amd64.deb ...
Unpacking libubsan1:amd64 (8.3.0-6) ...
Selecting previously unselected package libmpx2:amd64.
Preparing to unpack .../20-libmpx2_8.3.0-6_amd64.deb ...
Unpacking libmpx2:amd64 (8.3.0-6) ...
Selecting previously unselected package libquadmath0:amd64.
Preparing to unp

  Created wheel for tabulate: filename=tabulate-0.8.6-py3-none-any.whl size=23273 sha256=a699875683ae32cb6bb5094d30e4b8fa3cc7df366a20309340bfa76170237d39
  Stored in directory: /root/.cache/pip/wheels/09/b6/7e/08b4ee715a1239453e89a59081f0ac369a9036f232e013ecd8
  Created wheel for python-json-logger: filename=python_json_logger-0.1.11-py2.py3-none-any.whl size=5076 sha256=d09d98a0080952f6fae9a95e8ce53470205e1cd14b100859617cf40faeb51762
  Stored in directory: /root/.cache/pip/wheels/fa/7f/fd/92ccdbb9d1a65486406e0363d2ba5b4ce52f400a915f602ecb
  Created wheel for prometheus-client: filename=prometheus_client-0.7.1-py3-none-any.whl size=41402 sha256=21ab1ef311301f443503682364c128b48e8cc71eac3006b9c5e7b0803c123cc6
  Stored in directory: /root/.cache/pip/wheels/30/0c/26/59ba285bf65dc79d195e9b25e2ddde4c61070422729b0cd914
  Created wheel for alembic: filename=alembic-1.4.0-py2.py3-none-any.whl size=157563 sha256=c2458c589975c0cacb2b9e3800bb36f8f28c3985dc7d6097bdbd05c3bf3bd863
  Stored in direct

  Created wheel for BentoML: filename=BentoML-0.6.2+8.gd95a887-py3-none-any.whl size=513796 sha256=47aab287696ef482c10733fb11d3e1be5819f856756fb5f51e73d12a663cbcac
  Stored in directory: /root/.cache/pip/wheels/63/75/8b/c1e0439890f032f9c89af93138f58408c22c672822c9e60dd1
Successfully built BentoML
  Attempting uninstall: BentoML
    Found existing installation: BentoML 0.6.2
    Uninstalling BentoML-0.6.2:
      Successfully uninstalled BentoML-0.6.2
Removing intermediate container ee60bc4704bb
 ---> a95e372399a1
Step 12/12 : CMD ["bentoml serve-gunicorn /bento"]
 ---> Running in 8f2e9065cf3e
Removing intermediate container 8f2e9065cf3e
 ---> 38fa02890d9e
Successfully built 38fa02890d9e
Successfully tagged fastai-salary:latest


In [37]:
!docker run -p 5000:5000 fastai-salary

[2020-02-14 23:05:15,352] INFO - get_gunicorn_num_of_workers: 3, calculated by cpu count
[2020-02-14 23:05:15 +0000] [1] [INFO] Starting gunicorn 20.0.4
[2020-02-14 23:05:15 +0000] [1] [INFO] Listening at: http://0.0.0.0:5000 (1)
[2020-02-14 23:05:15 +0000] [1] [INFO] Using worker: sync
[2020-02-14 23:05:15 +0000] [9] [INFO] Booting worker with pid: 9
[2020-02-14 23:05:15 +0000] [10] [INFO] Booting worker with pid: 10
[2020-02-14 23:05:15 +0000] [11] [INFO] Booting worker with pid: 11
[2020-02-14 23:05:15,841] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.6.2,  but loading from BentoML version 0.6.2+8.gd95a887
[2020-02-14 23:05:15,880] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.6.2,  but loading from BentoML version 0.6.2+8.gd95a887
[2020-02-14 23:05:15,931] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with Bento

## Deployments

BentoML provides a set of APIs and CLI commands for automating cloud deployment workflow which gets your BentoService API server up and running in the cloud, and allows you to easily update and monitor the service. Currently BentoML have implemented this workflow for AWS Lambda, AWS Sagemaker and Azure Functions. More platforms such as AWS EC2, Kubernetes Cluster, Azure Virtual Machines are on our roadmap.

You can also manually deploy the BentoService API Server or its docker image to cloud platforms, and we’ve created a few step by step tutorials for doing that. You can visit those tutorials at BentoML documentation webiste, or click this [link](https://docs.bentoml.org/en/latest/deployment/index.html)